In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install seaborn

In [ ]:
!pip install torch

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:

def import_excel(file_path):
  # Load the Excel file
  xls = pd.ExcelFile(file_path)

  # Create a dictionary to store DataFrames
  dataframe = {}

  # Iterate through each sheet in the Excel file
  for sheet_name in xls.sheet_names:
      # Read each sheet into a DataFrame
      df = pd.read_excel(file_path, sheet_name=sheet_name)
      # Store the DataFrame with the sheet name as the key in the dictionary
      dataframe[sheet_name] = df
  return dataframe

# Now you have a dictionary where each key is the sheet name, and the value is the corresponding DataFrame
# For example, to access the DataFrame for the sheet named 'Sheet1':

df_phil = import_excel(r'c:\Users\phil_portal_data.xlsx')
print("df_phil sheets " + str(df_phil.keys()))
df_uganda = import_excel(r'c:\Users\uganda_portal_data.xlsx')
print("df_uganda sheets " + str(df_uganda.keys()))
df_nigeria = import_excel(r'c:\Users\nigeria_portal_data.xlsx')
print("df_nigeria sheets " + str(df_nigeria.keys()))
df_malawi = import_excel(r'c:\Users\malawi_portal_data.xlsx')
print("df_malawi sheets " + str(df_malawi.keys()))

# loop through the dictionary and convert to dataframe



In [ ]:
df_malawi.keys()

In [ ]:
df_uganda['ug_fact_numeracy_result'].head(100)

Comparing the availible columns and tables between dfs

Availible Tables(tables might have slightly differnt based on country):

'_fact_reading_result'

'_fact_student_registrations'

'_fact_student_attendance'

'_fact_schools'

'_fact_classes'

'_dim_students'


Extra tables:

Uganda: 'ug_fact_numeracy_result'

Nigeria: '_fact_student_reporting_rate'

In [ ]:
def compare_dfs(*dfs):
    # List to store common columns
    common_columns = None

    # Dictionary to store non-common columns for each DataFrame
    non_common_columns = {}

    # Iterate through each DataFrame
    for idx, df in enumerate(dfs):
        # If it's the first DataFrame, initialize common_columns with its columns
        if idx == 0:
            common_columns = set(df.columns)
        else:
            # Find the intersection of common columns and the columns of the current DataFrame
            common_columns &= set(df.columns)

        # Find the non-common columns for the current DataFrame
        non_common = set(df.columns) - common_columns
        #non_common_columns[f'df_{idx+1}'] = list(non_common)
        non_common_columns[f'{idx+1}'] = list(non_common)

    return list(common_columns), non_common_columns

# Example usage:
# Assuming df1, df2, and df3 are your pandas DataFrames
# common_cols, non_common_cols = compare_dfs(df1, df2, df3)
# print("Common Columns:", common_cols)
# print("Non-Common Columns:")
# for df_name, cols in non_common_cols.items():
#     print(f"DataFrame {df_name}: {cols}")


In [ ]:
dim_students_common_cols, dim_students_non_common_cols = compare_dfs(df_phil['_dim_students'],df_uganda['ug_dim_students'],df_nigeria['_dim_students'],df_malawi['_dim_students'])

In [ ]:
check = df_malawi['_dim_students']
check = check[check['Special_Needs'].notnull()]
df_malawi['_dim_students'].head()


In [ ]:
dim_students_common_cols

In [ ]:
dim_students_non_common_cols

Exploration:

"Special Needs" colum is an integer in ug only null in every other country what does the integer mean

In [ ]:
fact_reading_result_common_cols, fact_reading_result_non_common_cols = compare_dfs(df_phil['_fact_reading_result'],df_uganda['ug_fact_reading_result'],df_nigeria['_fact_reading_result'],df_malawi['_fact_reading_result'])

In [ ]:
df_phil['_fact_reading_result'].head()

In [ ]:
fact_reading_result_common_cols

In [ ]:
fact_reading_result_non_common_cols

In [ ]:
fact_student_registrations_common_cols, fact_student_registrations_non_common_cols = compare_dfs(df_phil['_fact_student_registrations'],df_uganda['ug_fact_student_registrations'],df_nigeria['_fact_student_registrations'],df_malawi['_fact_student_registrations'])

In [ ]:
fact_student_registrations_common_cols

In [ ]:
fact_student_registrations_non_common_cols

In [ ]:
fact_student_attendance_common_cols, fact_student_attendance_non_common_cols = compare_dfs(df_phil['_fact_student_attendance'],df_uganda['ug_fact_student_attendance'],df_nigeria['_fact_student_attendance'],df_malawi['_fact_student_attendance'])

In [ ]:
df_phil['_fact_student_attendance']

In [ ]:
fact_student_attendance_common_cols

In [ ]:
fact_student_attendance_non_common_cols

In [ ]:
fact_schools_common_cols, fact_schools_non_common_cols = compare_dfs(df_phil['_fact_schools'],df_uganda['ug_dim_schools'],df_nigeria['_fact_schools'],df_malawi['_dim_schools'])

In [ ]:
fact_schools_common_cols

In [ ]:
fact_schools_non_common_cols

In [ ]:
fact_classes_common_cols, fact_classes_non_common_cols = compare_dfs(df_phil['_fact_classes'],df_uganda['ug_fact_classes'],df_nigeria['_fact_classes'],df_malawi['_dim_classes'])

In [ ]:
fact_classes_common_cols

In [ ]:
fact_classes_non_common_cols

In [ ]:
def PrepData(df_students,df_reading_result,df_student_registrations,df_student_attendance,country_identifier):
  #clean and trim
  df_students = df_students[['ChildrenId','AgeCal','Gender','ParentType','DateOfBirth']]
  df_student_registrations = df_student_registrations[['ChildrenId','Grade']]
  df_reading_result = df_reading_result[['ChildrenId','ResultBaseline',"ResultRound1","ResultRound2","ResultRound3","ResultEndline"]]
  print(df_students.head())

  #attendance cols
  #total attendance rate
  attendance_rate = df_student_attendance.groupby('ChildrenId')['StudentIsPresent'].mean().reset_index()
  attendance_rate.rename(columns={'StudentIsPresent': 'AttendanceRate'}, inplace=True)
  #attendance rate at midpoint
  attendance_rate_at_midpoint = df_student_attendance.copy()
  attendance_rate_at_midpoint['midpoint_date'] = attendance_rate_at_midpoint.apply(lambda row: row['AcademicYearStartDate'] + (row['AcademicYearEndtDate'] - row['AcademicYearStartDate']) / 2, axis=1)
  attendance_rate_at_midpoint['StudentAttendanceDate'] = pd.to_datetime(attendance_rate_at_midpoint['StudentAttendanceDate'])
  attendance_rate_at_midpoint = attendance_rate_at_midpoint[attendance_rate_at_midpoint['StudentAttendanceDate'] <= attendance_rate_at_midpoint['midpoint_date']]
  attendance_rate_at_midpoint = attendance_rate_at_midpoint.groupby('ChildrenId')['StudentIsPresent'].mean().reset_index()
  attendance_rate_at_midpoint.rename(columns={'StudentIsPresent': 'AttendanceRateMidpoint'}, inplace=True)





  #merge
  df_students = pd.merge(df_students, df_student_registrations, on='ChildrenId', how='left')
  df_students = pd.merge(df_students, df_reading_result, on='ChildrenId', how='left')
  df_students = pd.merge(df_students, attendance_rate, on='ChildrenId', how='left')
  df_students = pd.merge(df_students, attendance_rate_at_midpoint, on='ChildrenId', how='left')

  #prep

  #test result cols
  df_students[['ResultBaseline',"ResultRound1","ResultRound2","ResultRound3","ResultEndline"]] = df_students[['ResultBaseline',"ResultRound1","ResultRound2","ResultRound3","ResultEndline"]].replace({"Beginner": 0.1,"Letter": 0.2,"Word": 0.3,"Sentence": 0.4,"Story": 0.5,"Story with comprehension": 0.6})
  #creating first result col (First non null test val)
  df_students['FirstResult'] = df_students['ResultBaseline'].combine_first(df_students['ResultRound1'])
  df_students['FirstResult'] = df_students['FirstResult'].combine_first(df_students['ResultRound2'])
  df_students['FirstResult'] = df_students['FirstResult'].combine_first(df_students['ResultRound3'])
  df_students['FirstResult'] = df_students['FirstResult'].combine_first(df_students['ResultEndline'])
  df_students['LastResult'] = df_students['ResultEndline'].combine_first(df_students['ResultRound3'])
  #Creating last result col (first non null val going back in time)
  df_students['LastResult'] = df_students['LastResult'].combine_first(df_students['ResultRound2'])
  df_students['LastResult'] = df_students['LastResult'].combine_first(df_students['ResultRound1'])
  df_students['LastResult'] = df_students['LastResult'].combine_first(df_students['ResultBaseline'])
  df_students[['FirstResult','LastResult','ResultBaseline',"ResultRound1","ResultRound2","ResultRound3","ResultEndline"]] = df_students[['FirstResult','LastResult','ResultBaseline',"ResultRound1","ResultRound2","ResultRound3","ResultEndline"]].fillna(0)
  df_students['ResultProgress'] = df_students['LastResult'] - df_students['FirstResult']


  df_students["Gender"] = df_students["Gender"].replace({'Male' : 0, 'Female' :1,'Unknown' :0.5}).fillna(0.5)
  df_students["ParentType"] = df_students["ParentType"].replace({'Father' : 0, 'Mother' :1, 'Guardian': 0.5}).fillna(0.5)
  df_students["AttendanceRate"] = df_students["AttendanceRate"].fillna(0)
  df_students["AttendanceRateMidpoint"] = df_students["AttendanceRateMidpoint"].fillna(0)
  df_students["AgeCal"] = df_students["AgeCal"].fillna(0)

  df_students['DateOfBirth'] = pd.to_datetime(df_students['DateOfBirth'])
  df_students['DateOfBirth'] = df_students['DateOfBirth'].dt.month.fillna(0)
  df_students.rename(columns={'DateOfBirth': 'MonthOfBirth'}, inplace=True)

  df_students['Grade'] = df_students['Grade'].str.extract('(\d+)').fillna(0).astype(int) * 0.1

  #make ChildrenID unique
  df_students['ChildrenId'] = df_students['ChildrenId'] + (country_identifier*1000000)
  print(df_students.head())
  return df_students,df_reading_result,df_student_registrations,df_student_attendance,attendance_rate


In [ ]:
#Staging area
#phil
#df_phil sheets dict_keys(['_fact_reading_result', '_fact_student_registrations', '_fact_student_attendance', '_fact_schools', '_fact_classes', '_dim_students'])
#initialize datasets
df_phil_students = df_phil['_dim_students']
df_phil_reading_result = df_phil['_fact_reading_result']
df_phil_student_registrations = df_phil['_fact_student_registrations']
df_phil_student_registrations.rename(columns={'ChildID': 'ChildrenId'}, inplace=True)
df_phil_student_attendance = df_phil['_fact_student_attendance']

df_phil_students,df_phil_reading_result,df_phil_student_registrations,df_phil_student_attendance,df_phil_attendance_rate = PrepData(df_phil_students,df_phil_reading_result,df_phil_student_registrations,df_phil_student_attendance,1)

#uganda
#df_uganda sheets dict_keys(['ug_fact_reading_result', 'ug_fact_student_registrations', 'ug_fact_student_attendance', 'ug_dim_students', 'ug_dim_schools', 'ug_fact_classes', 'ug_fact_numeracy_result'])
df_uganda_students = df_uganda['ug_dim_students']
df_uganda_reading_result = df_uganda['ug_fact_reading_result']
df_uganda_student_registrations = df_uganda['ug_fact_student_registrations']
df_uganda_student_registrations.rename(columns={'ChildID': 'ChildrenId'}, inplace=True)
df_uganda_student_attendance = df_uganda['ug_fact_student_attendance']

df_uganda_students,df_uganda_reading_result,df_uganda_student_registrations,df_uganda_student_attendance,df_uganda_attendance_rate = PrepData(df_uganda_students,df_uganda_reading_result,df_uganda_student_registrations,df_uganda_student_attendance,2)

#nigeria
#df_nigeria sheets dict_keys(['_fact_student_attendance', '_fact_schools', '_fact_classes', '_dim_students', '_fact_reading_result', '_fact_student_reporting_rate', '_fact_student_registrations'])
df_nigeria_students = df_nigeria['_dim_students']
df_nigeria_reading_result = df_nigeria['_fact_reading_result']
df_nigeria_student_registrations = df_nigeria['_fact_student_registrations']
df_nigeria_student_registrations.rename(columns={'ChildID': 'ChildrenId'}, inplace=True)
df_nigeria_student_attendance = df_nigeria['_fact_student_attendance']

df_nigeria_students,df_nigeria_reading_result,df_nigeria_student_registrations,df_nigeria_student_attendance,df_nigeria_attendance_rate = PrepData(df_nigeria_students,df_nigeria_reading_result,df_nigeria_student_registrations,df_nigeria_student_attendance,3)

#malawi
#df_malawi sheets dict_keys(['_fact_reading_result', '_fact_student_attendance', '_fact_student_registrations', '_dim_students', '_dim_schools', '_dim_classes'])

df_malawi_students = df_malawi['_dim_students']
df_malawi_reading_result = df_malawi['_fact_reading_result']
df_malawi_student_registrations = df_malawi['_fact_student_registrations']
df_malawi_student_registrations.rename(columns={'ChildID': 'ChildrenId'}, inplace=True)
df_malawi_student_attendance = df_malawi['_fact_student_attendance']

df_malawi_students,df_malawi_reading_result,df_malawi_student_registrations,df_malawi_student_attendance,df_malawi_attendance_rate = PrepData(df_malawi_students,df_malawi_reading_result,df_malawi_student_registrations,df_malawi_student_attendance,4)





In [ ]:
df_phil_students.head()

In [ ]:
df_phil_students.describe()

In [ ]:
df_uganda_students.head()

In [ ]:
df_uganda_students.describe()

In [ ]:
df_nigeria_students.head()

In [ ]:
df_nigeria_students.describe()

In [ ]:
df_malawi_students.head()

In [ ]:
df_malawi_students.describe()

In [ ]:
df_all_country_students = pd.concat([df_phil_students,df_uganda_students,df_nigeria_students,df_malawi_students]).drop(columns=['LastResult','ResultBaseline','ResultRound1','ResultRound2','ResultRound3','ResultEndline'])
df_all_country_students = df_all_country_students[df_all_country_students['ResultProgress'] >= 0]
#df_all_country_students['ResultProgress'] = df_all_country_students['ResultProgress'] * 10
#df_all_country_students['ResultProgress'] = df_all_country_students['ResultProgress'].astype(int)
df_all_country_students

In [ ]:
df_all_country_students = df_all_country_students.drop_duplicates()
df_all_country_students

In [ ]:
#count of unique students
df_all_country_students['ChildrenId'].nunique()

In [ ]:
# Count occurrences of each ChildrenId
child_id_counts = df_all_country_students['ChildrenId'].value_counts()

# Filter child_ids that occur more than once
duplicate_child_ids = child_id_counts[child_id_counts > 1].index

# Filter rows where child_id is in the list of duplicate_child_ids
duplicate_rows = df_all_country_students[df_all_country_students['ChildrenId'].isin(duplicate_child_ids)]

duplicate_rows

In [ ]:
#Remove rows where child appers multiple times with different data
df_all_country_students = df_all_country_students[~df_all_country_students['ChildrenId'].isin(duplicate_child_ids)]

In [ ]:
fig = px.histogram(df_all_country_students, x='ResultProgress', nbins=7, title='Distribution of Values')
fig

In [ ]:
df_all_country_students.ResultProgress.unique()

In [ ]:
#ML Stuff

#pre proccessing

X = df_all_country_students.drop(['ChildrenId','ResultProgress'], axis=1)
y = df_all_country_students['ResultProgress'].apply(lambda x: math.ceil(x))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
#Logistic Regression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test.apply(lambda x: math.ceil(x)))

In [ ]:
coef = clf.coef_[0]
odds = np.exp(clf.coef_[0])
pd.DataFrame(coef,
             pd.DataFrame(X_scaled,columns = X.columns).columns,
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

In [ ]:
pd.DataFrame(odds,
             X.columns,
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

In [ ]:
# Making predictions on the validation set
y_valid_pred = clf.predict(X_valid)
# Making predictions on the test data
y_test_pred = clf.predict(X_test)
# Calculating the accuracy of the model on the validation set
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
print("Validation Accuracy:", valid_accuracy)
# Calculating the accuracy of the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)


# Calculate the confusion matrix
y_true = y_test  # True labels (ground truth)
y_pred = y_test_pred  # Predicted labels from your model
confusion = confusion_matrix(y_true, y_pred)
TP, TN, FP, FN = confusion[1, 1], confusion[0, 0], confusion[0, 1], confusion[1, 0]
print("True Possitive: " + str(TP))
print("False Possitive: " + str(FP))
print("True Negative: " + str(TN))
print("False Negative: " + str(FN))

In [ ]:
# Plot confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# Creating a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Training the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Making predictions on the validation set
y_valid_pred = rf_classifier.predict(X_valid)

In [ ]:
# Calculating the accuracy of the model on the validation set
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
print("Validation Accuracy:", valid_accuracy)

In [ ]:
# Making predictions on the test data
y_test_pred = rf_classifier.predict(X_test)
# Calculating the accuracy of the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

In [ ]:
#
# Choose one of the decision trees from the forest (e.g., the first one)
tree_index = 0

# Plot the selected tree
#plt.figure()
#plot_tree(rf_classifier.estimators_[tree_index], feature_names=X.columns, class_names=['0', '1'], filled=True)
#plt.show()

In [ ]:
y_true = y_test  # True labels (ground truth)
y_pred = y_test_pred  # Predicted labels from your model

# Calculate the confusion matrix
confusion = confusion_matrix(y_true, y_pred)
TP, TN, FP, FN = confusion[1, 1], confusion[0, 0], confusion[0, 1], confusion[1, 0]
print("True Possitive: " + str(TP))
print("False Possitive: " + str(FP))
print("True Negative: " + str(TN))
print("False Negative: " + str(FN))

In [ ]:
# Plot confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:

feature_names = X.columns

importances = rf_classifier.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. Feature %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices], color="b", align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)  # Rotate labels for better visibility
plt.xlim([-1, X.shape[1]])
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.show()



In [ ]:
#Neural Net
class ReadingChangePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(ReadingChangePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Initialize the model
input_size = X_train.shape[1]
model = ReadingChangePredictionModel(input_size)

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 800
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(torch.Tensor(np.array(X_train)).float())
    loss = criterion(outputs.squeeze(), torch.Tensor(np.array(y_train.squeeze())).float())
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Evaluate your model on the test data
with torch.no_grad():
    test_inputs = torch.Tensor(np.array(X_valid)).float()
    test_outputs = model(test_inputs)
    predictions = (test_outputs > 0.5).float()
    y_test_df = pd.DataFrame({'ResultProgress': y_valid})
    y_test_tensor = torch.Tensor(y_test_df['ResultProgress'].values).view(-1, 1).float()
    accuracy = (predictions == y_test_tensor).sum().item() / len(y_test)
    print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
# Evaluate your model on the test data
with torch.no_grad():
    test_inputs = torch.Tensor(np.array(X_test)).float()
    test_outputs = model(test_inputs)
    predictions = (test_outputs > 0.5).float()
    y_test_df = pd.DataFrame({'ResultProgress': y_test})
    y_test_tensor = torch.Tensor(y_test_df['ResultProgress'].values).view(-1, 1).float()
    accuracy = (predictions == y_test_tensor).sum().item() / len(y_test)
    print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
y_true = y_test  # True labels (ground truth)
y_pred = predictions  # Predicted labels from your model

# Calculate the confusion matrix
confusion = confusion_matrix(y_true, y_pred)
TP, TN, FP, FN = confusion[1, 1], confusion[0, 0], confusion[0, 1], confusion[1, 0]

In [ ]:
print("True Possitive: " + str(TP))
print("False Possitive: " + str(FP))
print("True Negative: " + str(TN))
print("False Negative: " + str(FN))

In [ ]:
# Plot confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

Re-Doing analysis withoght total attendance

In [ ]:
#ML Stuff

#pre proccessing

X = df_all_country_students.drop(['ChildrenId','ResultProgress','AttendanceRate'], axis=1)
y = df_all_country_students['ResultProgress'].apply(lambda x: math.ceil(x))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
#Logistic Regression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf_score = clf.score(X_test, y_test.apply(lambda x: math.ceil(x)))
print("clf scroe is ", clf_score)
coef = clf.coef_[0]
odds = np.exp(clf.coef_[0])



In [ ]:
pd.DataFrame(coef,
             pd.DataFrame(X_scaled,columns = X.columns).columns,
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

In [ ]:
pd.DataFrame(odds,
             X.columns,
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

In [ ]:
# Creating a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=500, random_state=42)

# Training the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Making predictions on the validation set
y_valid_pred = rf_classifier.predict(X_valid)

In [ ]:
# Calculating the accuracy of the model on the validation set
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
print("Validation Accuracy:", valid_accuracy)

In [ ]:
# Making predictions on the test data
y_test_pred = rf_classifier.predict(X_test)
# Calculating the accuracy of the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
y_true = y_test  # True labels (ground truth)
y_pred = y_test_pred  # Predicted labels from your model

# Calculate the confusion matrix
confusion = confusion_matrix(y_true, y_pred)
TP, TN, FP, FN = confusion[1, 1], confusion[0, 0], confusion[0, 1], confusion[1, 0]
print("True Possitive: " + str(TP))
print("False Possitive: " + str(FP))
print("True Negative: " + str(TN))
print("False Negative: " + str(FN))

In [ ]:
# Plot confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:

feature_names = X.columns

importances = rf_classifier.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. Feature %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices], color="b", align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)  # Rotate labels for better visibility
plt.xlim([-1, X.shape[1]])
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.show()

In [ ]:
#Neural Net
class ReadingChangePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(ReadingChangePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Initialize the model
input_size = X_train.shape[1]
model = ReadingChangePredictionModel(input_size)

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 800
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(torch.Tensor(np.array(X_train)).float())
    loss = criterion(outputs.squeeze(), torch.Tensor(np.array(y_train.squeeze())).float())
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Evaluate your model on the test data
with torch.no_grad():
    test_inputs = torch.Tensor(np.array(X_valid)).float()
    test_outputs = model(test_inputs)
    predictions = (test_outputs > 0.5).float()
    y_test_df = pd.DataFrame({'ResultProgress': y_valid})
    y_test_tensor = torch.Tensor(y_test_df['ResultProgress'].values).view(-1, 1).float()
    accuracy = (predictions == y_test_tensor).sum().item() / len(y_test)
    print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
# Evaluate your model on the test data
with torch.no_grad():
    test_inputs = torch.Tensor(np.array(X_test)).float()
    test_outputs = model(test_inputs)
    predictions = (test_outputs > 0.5).float()
    y_test_df = pd.DataFrame({'ResultProgress': y_test})
    y_test_tensor = torch.Tensor(y_test_df['ResultProgress'].values).view(-1, 1).float()
    accuracy = (predictions == y_test_tensor).sum().item() / len(y_test)
    print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
y_true = y_test  # True labels (ground truth)
y_pred = predictions  # Predicted labels from your model

# Calculate the confusion matrix
confusion = confusion_matrix(y_true, y_pred)
TP, TN, FP, FN = confusion[1, 1], confusion[0, 0], confusion[0, 1], confusion[1, 0]

In [ ]:
print("True Possitive: " + str(TP))
print("False Possitive: " + str(FP))
print("True Negative: " + str(TN))
print("False Negative: " + str(FN))

In [ ]:
# Plot confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()